In [20]:
class Indivudal:
    def __init__(self, cost, fixed_cost):
        self.code = [random.random() < 0.25 for _ in range(len(fixed_cost))]
        self.fitness = self.calc_fitness(cost, fixed_cost)
        
#     def __lt__(self, other):
#         return self.fitness < other.fitness
    
    def calc_fitness(self, cost, fixed_cost):
        if sum(self.code) == 0:
            return 0
        
        fitness = 0
        num_users = len(cost)
        num_resources = len(fixed_cost)
        for i in range(num_users):
#             min_cost = float('inf')
#             for j in range(num_resources):
#                 if self.code[j] and cost[i][j] < min_cost:
#                     min_cost = cost[i][j]
#             fitness += min_cost
            fitness += min(map(lambda pair: pair[1],
                            filter(lambda pair: self.code[pair[0]],
                                   enumerate(cost[i])
                                  )
                              )
                          )

#         for j in range(num_resources):
#             if self.code[j]:
#                 fitness += fixed_cost[j]
        fitness += sum(
                        map(
                            lambda pair: pair[1],
                            filter(
                                lambda pair: self.code[pair[0]],
                                enumerate(fixed_cost)
                                  )
                              )
                      )

        return 1 / fitness
        
#         drugi nacin
#         najgori slucaj - saberemo sve iz cost i fixed_cost
#         return -fitness

In [21]:
def read_instance(file_path):
    with open(file_path, 'r') as f:
        num_users, num_resources = [int(x) for x in f.readline().split()]
        cost = [[int(x) for x in f.readline().split()] for i in range(num_users)]
        fixed_cost = [int(x) for x in f.readline().split()]
        return cost, fixed_cost

In [22]:
import random

In [23]:
cost, fixed_cost = read_instance('../03/uflp1.txt')

In [24]:
def selection(population, tournament_size, forbidden):
    allowed = list(set(population).difference({forbidden}))
    chosen = random.sample(allowed, tournament_size)
    max_fitness = float('-inf')
    best_idx = -1
    for idx, i in enumerate(chosen):
        if i.fitness > max_fitness:
            max_fitness = i.fitness
            best_idx = idx
    return best_idx

In [33]:
def mutation(child, mutation_prob):
    for i in range(len(child.code)):
        if random.random() < mutation_prob:
            child.code[i] = not child.code[i]

In [30]:
def crossover(parent1, parent2, child1, child2):
    pos = random.randrange(1, len(parent1.code))
    
    child1.code[:pos] = parent1.code[:pos]
    child1.code[pos:] = parent2.code[pos:]
    
    child2.code[:pos] = parent2.code[:pos]
    child2.code[pos:] = parent1.code[pos:]

In [41]:
def ga(pop_size, num_iters, tournament_size, mutation_prob, use_elitism, elitism_size):
    if (pop_size - elitism_size) % 2 == 1:
        elitism_size += 1
    
    population = [Indivudal(cost, fixed_cost) for _ in range(pop_size)]
    new_population = [Indivudal(cost, fixed_cost) for _ in range(pop_size)]
    for _ in range(num_iters):
        if use_elitism:
            population.sort(key=lambda x: x.fitness, reverse=True)
            
            new_population[:elitism_size] = population[:elitism_size]
            
        for i in range(elitism_size, pop_size, 2):
            parent1_idx = selection(population, tournament_size, None)
            parent2_idx = selection(population, tournament_size, parent1_idx)
            
            crossover(population[parent1_idx],
                      population[parent2_idx],
                      new_population[i],
                      new_population[i+1])
            
            mutation(new_population[i], mutation_prob)
            mutation(new_population[i+1], mutation_prob)
            
            
        
        population[:] = new_population[:]
    
    best_individual = max(population, key=lambda x: x.fitness)
    print(best_individual.code)
    print(best_individual.fitness)
    print(1 / best_individual.fitness)

In [42]:
ga(100, 10, 5, 0.05, True, 19)

[True, False, False]
0.029411764705882353
34.0
